In [1]:
import botocore
from boto3.session import Session
import time
import throttle

# 准备密钥
aws_access_key_id = 'hust'
aws_secret_access_key = 'hust_obs'

# 建立会话
session = Session(aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

# 连接到服务
s3 = session.resource('s3', endpoint_url='http://192.168.33.1:9000')

查看所有bucket

In [2]:
for bucket in s3.buckets.all():
    print('bucket name:%s' % bucket.name)

bucket name:loadgen


新建一个bucket (注意："bucket name" 中不能有下划线)

In [4]:
bucket_name = 'test100objs'
if s3.Bucket(bucket_name) not in s3.buckets.all():
    s3.create_bucket(Bucket=bucket_name)

查看此bucket下的所有object

In [5]:
bucket = s3.Bucket(bucket_name)
for obj in bucket.objects.all():
    print('obj name:%s' % obj.key)

准备负载，按照几种不同请求到达率(Inter-Arrival Time, IAT)设置。

In [6]:
# 发起请求和计算系统停留时间
def request_timing(i):
    # 上传obj
    obj_name = "testObj%08d"%(i,)
    local_file = 'README.md'
    # temp_file = '.tempfile'
    service_time = 0
    start = time.time()
    s3.Object(bucket_name, obj_name).upload_file(local_file)
    # 或
    # bucket.put_object(Key=obj_name, Body=open(local_file, 'rb'))
    # 下载obj
    # s3.Object(bucket_name, obj_name).download_file(temp_file)
    end = time.time()
    system_time = end - start
    return system_time * 1000 # 换算为毫秒

# 按照请求到达率限制来执行和跟踪请求
def arrival_rate_max(i): # unlimited
    return request_timing(i)

@throttle.wrap(0.1, 2) # 100ms 2 个
def arrival_rate_2(i):
    return request_timing(i)

@throttle.wrap(0.1, 4)
def arrival_rate_4(i):
    return request_timing(i)

@throttle.wrap(0.1, 8)
def arrival_rate_8(i):
    return request_timing(i)

按照预设IAT发起请求

In [7]:
latency = []
for i in range(100):
    # 上传obj
    st = arrival_rate_max(i)
    succ = not st is throttle.fail
    print(succ, st)
    if succ:
        latency.append(st)

True 25.860309600830078
True 16.825437545776367
True 13.784170150756836
True 23.554563522338867
True 22.9341983795166
True 12.856483459472656
True 15.47694206237793
True 22.40443229675293
True 26.862144470214844
True 14.463186264038086
True 15.209436416625977
True 15.250444412231445
True 13.782024383544922
True 15.175819396972656
True 14.690637588500977
True 24.734973907470703
True 26.947975158691406
True 19.07038688659668
True 18.70894432067871
True 15.949726104736328
True 14.002323150634766
True 13.886451721191406
True 13.472795486450195
True 12.658119201660156
True 15.999078750610352
True 16.99066162109375
True 18.594026565551758
True 21.39759063720703
True 15.786409378051758
True 52.542686462402344
True 14.214277267456055
True 13.426065444946289
True 13.730525970458984
True 14.903068542480469
True 16.353368759155273
True 19.608020782470703
True 24.484634399414062
True 13.946533203125
True 13.319730758666992
True 24.176597595214844
True 24.076461791992188
True 18.31674575805664
True

清理

In [8]:
try:
    # 删除bucket下所有object
    bucket.objects.filter().delete()

    # 删除bucket下某个object
    # bucket.objects.filter(Prefix=obj_name).delete()

    bucket.delete()
except botocore.exceptions.ClientError as e:
    print('error in bucket removal')

记录延迟到CSV文件

In [9]:
with open("latency.csv", "w+") as tracefile:
    tracefile.write("latency\n")
    tracefile.writelines([str(l) + '\n' for l in latency])